# Craigslist Vehicle Listing Dataset

### Exploratory Data Analysis & the ETL Process

---

### Objective

The goal of this project is to 

<b>The steps will be as follows:</b>

EXTRACT data from a database of Craigslist vehicle postings found on Kaggle into a Pandas DataFrame.

(Source: https://www.kaggle.com/austinreese/craigslist-carstrucks-data)

Clean and TRANSFORM the data into a smaller, more useful and manageable set in order to extract insights and visualize trends.

Use SQLAlchemy and PostGreSQL to LOAD the dataset into a database. 

Also, save the cleaned dataset as a .csv file for use in future projects.

---

### Description of Data

 This data set consists of all vehicles listed to Craigslist in the US. It contains most all relevant information that Craigslist provides on car sales including columns like price, condition, manufacturer, latitude/longitude, and 18 other categories.
 
 ---


<b> Import Dependencies </b>

In [1]:
import pandas as pd
import numpy as np

### Extract & Explore Data

In [19]:
# Read the data from csv and put it in a dataframe

raw_df = pd.read_csv("../Resources/vehicles.csv")

raw_df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7184791621,https://duluth.craigslist.org/ctd/d/duluth-200...,duluth / superior,https://duluth.craigslist.org,6995,2000.0,gmc,new sierra 1500,excellent,8 cylinders,...,4wd,NaN,NaN,red,https://images.craigslist.org/00n0n_f06ykBMcdh...,2000 *** GMC New Sierra 1500 Ext Cab 157.5 WB...,NaN,mn,46.8433,-92.2550
1,7184773187,https://duluth.craigslist.org/cto/d/saginaw-20...,duluth / superior,https://duluth.craigslist.org,8750,2013.0,hyundai,sonata,excellent,4 cylinders,...,fwd,NaN,NaN,grey,https://images.craigslist.org/00d0d_kgZ6xoeRw2...,For Sale: 2013 Hyundai Sonata GLS - $8750. O...,NaN,mn,46.9074,-92.4638
2,7193375964,https://newhaven.craigslist.org/cto/d/stratfor...,new haven,https://newhaven.craigslist.org,10900,2013.0,toyota,prius,good,4 cylinders,...,fwd,NaN,NaN,blue,https://images.craigslist.org/00d0d_3sHGxPbY2O...,2013 Prius V Model Two. One owner—must sell my...,NaN,ct,41.1770,-73.1336
3,7195108810,https://albuquerque.craigslist.org/cto/d/albuq...,albuquerque,https://albuquerque.craigslist.org,12500,2003.0,mitsubishi,lancer,good,4 cylinders,...,4wd,mid-size,sedan,grey,https://images.craigslist.org/00m0m_4a8Pb6JbMG...,"2003 Mitsubishi Lancer Evolution, silver. Abo...",NaN,nm,35.1868,-106.6650
4,7184712241,https://duluth.craigslist.org/ctd/d/rush-city-...,duluth / superior,https://duluth.craigslist.org,16995,2007.0,gmc,sierra classic 2500hd,good,8 cylinders,...,4wd,full-size,truck,white,https://images.craigslist.org/01414_g093aPtSMW...,"**Bad Credit, No Credit... No Problem!**2007 G...",NaN,mn,45.6836,-92.9648


In [21]:
# explore column names

raw_df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

<b>The data set has 25 columns, but we will not need all  of them for this analysis.</b>

Since my goal is to practice SQL, I will turn this single dataset into a relational database, using the ID column as the primary key.

- Listings table: includes all the listing information that does not have to do with the location.
- Location table: includes region, county, state, latitude, and longitude columns.
- Description table: includes the descriptions for each listing
    
In order to mimic a project in which there are multiple data sources, I will separate the dataset into three different tables.
I will perform the transformation process on just the main Listings table, and will not touch the other two tables.
Once I have completed the bulk of the data transformation, I will join the data tables.
    
<b>Upon initial exploration, I do not see a use-case for the URL or VIN columns, so they will be dropped.</b>






In [22]:
raw_df.drop(['url','region_url','vin','image_url'], axis=1, inplace=True)
raw_df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,transmission,drive,size,type,paint_color,description,county,state,lat,long
0,7184791621,duluth / superior,6995,2000.0,gmc,new sierra 1500,excellent,8 cylinders,gas,167783.0,...,automatic,4wd,NaN,NaN,red,2000 *** GMC New Sierra 1500 Ext Cab 157.5 WB...,NaN,mn,46.8433,-92.2550
1,7184773187,duluth / superior,8750,2013.0,hyundai,sonata,excellent,4 cylinders,gas,90821.0,...,automatic,fwd,NaN,NaN,grey,For Sale: 2013 Hyundai Sonata GLS - $8750. O...,NaN,mn,46.9074,-92.4638
2,7193375964,new haven,10900,2013.0,toyota,prius,good,4 cylinders,hybrid,92800.0,...,automatic,fwd,NaN,NaN,blue,2013 Prius V Model Two. One owner—must sell my...,NaN,ct,41.1770,-73.1336
3,7195108810,albuquerque,12500,2003.0,mitsubishi,lancer,good,4 cylinders,gas,NaN,...,manual,4wd,mid-size,sedan,grey,"2003 Mitsubishi Lancer Evolution, silver. Abo...",NaN,nm,35.1868,-106.6650
4,7184712241,duluth / superior,16995,2007.0,gmc,sierra classic 2500hd,good,8 cylinders,diesel,254217.0,...,automatic,4wd,full-size,truck,white,"**Bad Credit, No Credit... No Problem!**2007 G...",NaN,mn,45.6836,-92.9648


In [25]:
# Inspect the remaining columns
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423857 entries, 0 to 423856
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            423857 non-null  int64  
 1   region        423857 non-null  object 
 2   price         423857 non-null  int64  
 3   year          328743 non-null  float64
 4   manufacturer  313242 non-null  object 
 5   model         325384 non-null  object 
 6   condition     176719 non-null  object 
 7   cylinders     197679 non-null  object 
 8   fuel          327214 non-null  object 
 9   odometer      270585 non-null  float64
 10  title_status  327759 non-null  object 
 11  transmission  328065 non-null  object 
 12  drive         231119 non-null  object 
 13  size          102627 non-null  object 
 14  type          241157 non-null  object 
 15  paint_color   222203 non-null  object 
 16  description   329654 non-null  object 
 17  county        0 non-null       float64
 18  stat

In [27]:
listings_df = pd.DataFrame(raw_df['id','price','year','manufacturer','model','condition','cylinders','fuel','odometer','title_status','transmission','drive','size','type','paint_color'])
listings_df.head()

KeyError: ('id', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color')

0    7184791621
1    7184773187
2    7193375964
3    7195108810
4    7184712241
Name: id, dtype: int64